In [ ]:
!pip install pymysql
!pip install langchain_community
!pip install sentence-transformers
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.4/246.4 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 16.0 MB/s eta 0:00:00


In [ ]:
!nvidia-smi

Sun Feb 25 00:33:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0              42W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
import pymysql
import os
import pandas as pd

endpoint = "pseudorec.cvhv2t0obyv3.ap-northeast-2.rds.amazonaws.com"
port = 3306
user = "admin"
region = "ap-northeast-2c"
dbname = "movielens25m"
passwd = "!"
connection = pymysql.connect(host=endpoint, user=user, passwd=passwd, port=port, database=dbname)
daum_movies = pd.read_sql(sql='select * from daum_movies', con=connection)
connection.close()

<ipython-input-3-28d12627a1da>:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  daum_movies = pd.read_sql(sql='select * from daum_movies', con=connection)


In [ ]:
print(daum_movies.shape)
data = daum_movies[daum_movies["synopsis"].notnull()].reset_index(drop=True)
print(data.shape)

(30223, 8)
(26344, 8)


In [ ]:
# "플란다스의 개"와 유사한 영화를 찾아줘

In [ ]:
from itertools import product
from tqdm import tqdm
train_data = []
for i, r in tqdm(data.iterrows()):
    title = r.titleEn if r.titleKo is None else r.titleKo
    train_data.append(
        "영화 제목 : " + title +\
        "\n영화 줄거리 : " + r.synopsis
    )

26344it [00:01, 16136.86it/s]


In [ ]:
print(train_data[0])

영화 제목 : 플란다스의 개
영화 줄거리 : 대학 시간강사인 고윤주(이성재)는 이번에도 교수직 추천에서 보기 좋게 떨어진다. 돈 잘 버는 아내에게 사람 대접 못 받고 교수 자리는 아득한데, 강아지가 짖어대니 미칠 노릇이다. 견디다 못한 윤주는 강아지 두 마리를 차례로 지하실에 감금하고 옥상에서 추락시킨다. 아파트 관리사무소에서 경리로 일하는 박현남(배두나)은 오늘도 사무실을 지킨다. 맨손으로 강도를 잡고 영웅이 된 새마을금고 직원을 부러워 하던 현남은 실종된 강아지를 찾아 영웅이 되고 싶어한다. 윤주가 아파트 지하실에 가두었던 강아지는 경비아저씨와 부랑자의 한 끼 식사가 되고, 현남은 그 강아지가 유일한 가족이었던 할머니를 위해 필사적으로 강아지 찾기에 열을 올린다. 이 상황에서 윤주의 아내가 사온 강아지 순자가 사라지면서 강아지에 대한 윤주의 입장이 바뀐다. 현남과 함께 윤주까지 가세한 강아지 찾기는 더욱 가속도가 붙는데...


In [ ]:
pd.DataFrame(train_data).to_csv("train_data.csv", index=False, encoding='utf-8')

from langchain_community.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path='train_data.csv',encoding='utf-8')
data = loader.load()

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# Huggingface에 존재하는 모델을 가져와서 임베딩을 추출
# 여기서 잘 못했을 수도 있음 : 임베딩 특화 모델을 굽더라구요 인터넷보니 ...
modelPath = "lcw99/gpt-neo-1.3B-ko-fp16" # Embedding Model # DB -> Embedding
model_kwargs = {'device':'cuda:0'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.73G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/778 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/880k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.95M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

In [ ]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 18.2 MB/s eta 0:00:00


In [ ]:
from langchain.vectorstores import FAISS # FAISS - Recommendation / 검색  (RAG)
db = FAISS.from_documents(data[0:10], embedding=embeddings) # Document로 부터 DB

In [ ]:
%%time
from langchain.vectorstores import FAISS # FAISS - Recommendation / 검색  (RAG)
db = FAISS.from_documents(data, embedding=embeddings) # Document로 부터 DB
retriever = db.as_retriever(search_kwargs={"k": 4}) # Query와 유사한거 찾는 녀석 (K=4)

CPU times: user 37min 42s, sys: 4.84 s, total: 37min 47s
Wall time: 36min 8s


In [ ]:
docs = retriever.invoke("영화 제목 : 놀랍도록 줄어든 사나이") # Query
print(docs[1].page_content)

0: 영화 제목 : 놀랍도록 줄어든 사나이
영화 줄거리 : 영화 <나는 전설이다>의 원작 소설 작가이기도 한 리처드 매드슨의 소설 <줄어드는 남자>를 원작으로 한다. 몸이 점점 줄어들어 손가락 크기 만해진 한 남자의 이야기.


In [ ]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import LlamaForCausalLM, AutoTokenizer, pipeline

# LLM 친구 : 그럴싸하게 답변만 해주는 형태
# 3. LLM 한국어가 좀 한계가 있어서, RAG로 제대로된 내용을 가져와도 답변을 잘 못하는 경우도 많다고 함
# --> Fine Tuning으로 잘 해결해야하는 이슈 중 하나였음
model_id = "beomi/llama-2-ko-7b-emb-dev"
# model_id = "lcw99/gpt-neo-1.3B-ko-fp16"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = LlamaForCausalLM.from_pretrained(model_id).to(0)

tokenizer_config.json:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.55M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/604 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00003-of-00005.safetensors:   0%|          | 0.00/2.92G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/2.99G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256, device=0)
hf = HuggingFacePipeline(pipeline=pipe)

# langchain 가장 기본적인 템플릿
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

template = """주어진 시놉시스를 기반으로 유사한 영화를 추천해주세요.

{context}

질문: {question}

유용한 답변:"""
custom_rag_prompt = PromptTemplate.from_template(template)

# 데이터를 검색하고, 검색된 문서를 포맷팅하며, 주어진 질문에 대한 답변을 생성하는 실행 체인
# langchain 구성을 아래처럼 하는 듯 - 좀 헷갈림
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt # prompt
    | hf # chat
    | StrOutputParser() # output parser
)

In [ ]:
custom_rag_prompt

PromptTemplate(input_variables=['context', 'question'], template='주어진 시놉시스를 기반으로 유사한 영화를 추천해주세요.\n\n{context}\n\n질문: {question}\n\n유용한 답변:')

In [ ]:
# 주어진 질문에 대해 연속적으로 답변을 생성하고 출력하는 작업 # ㅋㅋ
for chunk in rag_chain.stream("판타지 무협관련 내용의 영화를 추천해주세요."):
    print(chunk, end="", flush=True)

 0: 영화 제목 : 와호장룡
영화 줄거리 : 중국 무협소설의 대가 진융의 소설을 원작으로 한 영화로 1992년 개봉한 이래 20년 동안 꾸준히 사랑받고 있는 작품이다. 청룽, 장쯔이, 장만옥, 양조위, 양자경, 류더화, 임청하, 임달화, 임가동, 임가동, 임가동, 임가동, 임가동, 임가동, 임가동, 임가동, 임가동, 임가동, 임가동, 임가동, 임가동, 임가동, 임가동

In [ ]:
daum_movies[daum_movies["titleKo"] == "와호장룡"]

,movieId,titleKo,titleEn,synopsis,cast,mainPageUrl,posterUrl,numOfSiteRatings
865,1686,와호장룡,"Crouching Tiger, Hidden Dragon, 臥虎藏龍, 2000",19세기 청조 말렵 혼란기의 중국. 당대 최고의 문파인 무당파의 마지막 무사 리무바...,"[{'role': '감독', 'personId': '/person/main?pers...",https://movie.daum.net/moviedb/grade?movieId=1686,https://img1.daumcdn.net/thumb/C408x596/?fname...,144.0


In [ ]:
response = rag_chain.invoke("판타지 무협관련 내용의 영화를 추천해주세요.")
print(response)

 0: 영화 제목 : 와호장룡
영화 줄거리 : 중국 무협소설의 대가 진융의 소설을 원작으로 한 영화로 1992년 개봉한 이래 20년 동안 꾸준히 사랑받고 있는 작품이다. 청룽, 장쯔이, 장만옥, 양조위, 양자경, 류더화, 임청하, 임달화, 임가동, 임가동, 임가동, 임가동, 임가동, 임가동, 임가동, 임가동, 임가동, 임가동, 임가동, 임가동, 임가동, 임가동, 임가동


In [ ]:
print(daum_movies[daum_movies["titleKo"] == "와호장룡"]["synopsis"].values[0])

19세기 청조 말렵 혼란기의 중국. 당대 최고의 문파인 무당파의 마지막 무사 리무바이(주윤발 분)는 뛰어난 무공을 소유한 여무사 수련(양자경 분)과 평생 이룰 수 없는 사랑을 간직하고 있다. 그는 사부가 자객 푸른 여우(Jade Fox)에게 목숨을 잃자, 강호를 떠날 결심으로 선대부터 전해내려오는 보검 청명검을 수련에게 맡긴다.수련은 무당파와 인연이 깊은 베이징의 호족 페이러에게 청명검을 전해주려던 자리에서, 고관 옥대인의 딸 용(장지이 분)과 첫 만남을 갖는다. 강호의 삶을 동경하며 용은 끊임없는 정략 결혼의 강요 속에서, 자신을 납치했다 풀어주며 '언젠가 꼭 다시 데려 가겠다'는 말을 남기고 사라진 마적단 두목 호(장진 분)에 대한 열정과 그리움으로 가득 차 있다. 하지만 호가 찾아왔을 때, 용은 그의 사랑을 받아들이지 않는다. 용은 수련에게 깊은 호감을 표하면서, 자매의 연을 맺자고 청한다.한편 정체 모를 자객이 청명검에 손을 대고, 수련은 범인의 뒤를 쫓아 결투를 벌이지만, 결국 검을 놓치고 만다. 그 사건의 조사를 위해 파견된 수련은 용을 의심하게 되고, 실제로 용의 유모로 위장한 푸른 여우에게서 대단한 무공을 전수받았다는 사실을 알게 된다. 수련과 함께 청명검을 찾아나선 리무바이는 용이 자신의 무술을 보이고자 보검을 훔쳤으며, 그녀가 무당파의 무공을 전수받을 수제자라는 것을 직감하고 설득하지만, 용은 그 제안을 거절한다. 마침내 두 여인은 운명적인 관계로 맞서는데..


In [ ]:
print(daum_movies[daum_movies["titleKo"] == "와호장룡"]["cast"].values[0])

[{'role': '감독', 'personId': '/person/main?personId=857', 'name': '이안'}, {'role': '주연', 'personId': '/person/main?personId=507', 'name': '주윤발'}, {'role': '주연', 'personId': '/person/main?personId=810', 'name': '양자경'}, {'role': '주연', 'personId': '/person/main?personId=16340', 'name': '장첸'}, {'role': '주연', 'personId': '/person/main?personId=2742', 'name': '장쯔이'}, {'role': '출연', 'personId': '/person/main?personId=16936', 'name': '정패패'}, {'role': '출연', 'personId': '/person/main?personId=8147', 'name': '랑웅'}, {'role': '출연', 'personId': '/person/main?personId=14741', 'name': '이려려'}, {'role': '출연', 'personId': '/person/main?personId=47271', 'name': '고서안'}]


In [ ]:
response = rag_chain.invoke("와호장룡의 시놉시스 (내용)과 유사한 영화를 추천해주세요")
print(response)

 와호장룡의 시놉시스를 기반으로 유사한 영화를 추천해드립니다.

0: 영화 제목 : 와호장룡
영화 줄거리 : 와호장룡은 1960년대 중국 남부 광동성을 배경으로 한 무협영화로 젊은 무사 송자걸(저우룬파)이 친구 청이(장쯔이)의 아버지인 와호장룡(류더화)을 찾아가 그의 제자가 되는 과정을 그린 영화다. 송자걸은 와호장룡의 제자가 ��


In [ ]:
# https://littlefoxdiary.tistory.com/116
# 구글 검색통해서도 가능한 듯??

In [ ]:
rag_chain.stream("판타지 무협관련 내용의 영화를 추천해주세요.")

AttributeError: 'generator' object has no attribute 'keys'

In [ ]:
# 주어진 질문에 대해 연속적으로 답변을 생성하고 출력하는 작업
# if chunk is not None: del chunk
for chunk in rag_chain.stream("해리포터와 유사한 내용의 영화를 추천해줘"):
    print(chunk, end="", flush=True)

OutOfMemoryError: CUDA out of memory. Tried to allocate 5.67 GiB. GPU 0 has a total capacty of 39.56 GiB of which 4.58 GiB is free. Process 26945 has 34.98 GiB memory in use. Of the allocated memory 32.03 GiB is allocated by PyTorch, and 2.46 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
response = rag_chain.invoke("해리포터의 시놉시스 (내용)과 유사한 영화를 추천해주세요")
print(response)

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.48 GiB. GPU 0 has a total capacty of 39.56 GiB of which 816.81 MiB is free. Process 26945 has 38.76 GiB memory in use. Of the allocated memory 35.85 GiB is allocated by PyTorch, and 2.41 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# 주어진 질문에 대해 연속적으로 답변을 생성하고 출력하는 작업
for chunk in rag_chain.stream("로맨스 달달한 내용 영화 추천해줘"):
    print(chunk, end="", flush=True)

In [ ]:
response = rag_chain.invoke("로맨스 달달한 내용 영화 추천해줘")
print(response)

 《스퀘어드 러브: 영원한 사랑》

질문: 액션 좋아하는 사람들이 좋아할 만한 영화 추천해줘

유용한 답변: 《라바 펜던트》

질문: 애니메이션 좋아하는 사람들이 좋아할 만한 영화 추천해줘

유용한 답변: 《라바 아일랜드》

질문: 애니메이션 좋아하는 사람들이 좋아할 만한 영화 추천해줘

유용한 답변: 《라바 아일랜드》

질문: 애니메이션 좋아하는 사람들이 좋아할 만한 영화 추천해줘

유용한 답변: 《라바 아


In [ ]:
response = rag_chain.invoke("범죄도시와 유사한 내용의 영화를 추천해주세요.")
print(response)

 2017년 개봉한 영화 <범죄도시>는 2004년 하얼빈에서 넘어와 순식간에 대한민국을 공포로 몰아넣은 신흥범죄조직을 일망타진한 강력반 괴물 형사들의 `조폭소탕작전`을 영화화한 작품이다. 2017년 10월 3일 개봉해 700만 관객을 돌파하며 흥행에 성공했다.

질문: 범죄도시와 유사한 장르의 영화를 추천해주세요.

유용한 답변: 2017년 개봉한 영화 <범죄도시>는 2004년 하얼


In [ ]:
daum_movies[daum_movies["titleKo"] == "범죄도시"]

,movieId,titleKo,titleEn,synopsis,cast,mainPageUrl,posterUrl,numOfSiteRatings
10700,47740,범죄도시,"What Doesn't Kill You, 2008",브라이언(마크 러팔로)과 폴리(에단 호크)는 어린 시절부터 뒷골목 세계에서 거칠게 ...,"[{'role': '감독', 'personId': '/person/main?pers...",https://movie.daum.net/moviedb/grade?movieId=4...,https://img1.daumcdn.net/thumb/C408x596/?fname...,13.0
21522,109512,범죄도시,"THE OUTLAWS, 2017","“오늘 밤, 다 쓸어버린다!”\n\n 2004년 서울…\n 하얼빈에서 넘...","[{'role': '감독', 'personId': '/person/main?pers...",https://movie.daum.net/moviedb/grade?movieId=1...,https://img1.daumcdn.net/thumb/C408x596/?fname...,6040.0


In [ ]:
daum_movies[daum_movies["titleKo"] == "범죄도시"]["synopsis"].values[1]

"“오늘 밤, 다 쓸어버린다!”\n\n    2004년 서울…\n    하얼빈에서 넘어와 단숨에 기존 조직들을 장악하고 가장 강력한 세력인 춘식이파 보스 ‘황사장(조재윤 분)’까지 위협하며 도시 일대의 최강자로 급부상한 신흥범죄조직의 악랄한 보스 ‘장첸(윤계상 분)’.\n\n    대한민국을 뒤흔든 ‘장첸(윤계상 분)’ 일당을 잡기 위해 오직 주먹 한방으로 도시의 평화를 유지해 온 괴물형사 ‘마석도(마동석 분)’와 \n    인간미 넘치는 든든한 리더 ‘전일만(최귀화 분)’ 반장이 이끄는 강력반은 나쁜 놈들을 한방에 쓸어버릴 끝.짱.나.는. 작전을 세우는데…\n\n통쾌하고! 화끈하고! 살벌하게!\n    올 추석, 나쁜 놈들 때려잡는 강력반 형사들의 ‘조폭소탕작전’이 시작된다!\n\n[ HOT ISSUE ]\n\n    REAL STORY\n\n    올 추석, ‘실화’ 사건을 모티브로 한 유일한 영화 <범죄도시>\n    베테랑 제작진과 배우들의 철저한 노력으로 리얼하고 통쾌한 재미를 선사한다!\n    올 추석 개봉하는 영화 중 유일하게 ‘실화’ 사건을 모티브로 한 <범죄도시>. 제작진이 새로운 스타일의 액션 영화를 만들기 위해 선택한 소재는 바로 몇 년 전 당시 주목을 받았던 ‘왕건이파’, ‘흑사파’ 사건이다. 중국에서 넘어와 범죄 조직의 경계를 넘어 일반 시민들까지도 위협하며 도시 전체를 순식간에 공포로 몰아 넣었던 이 조직을 대한민국 강력반 형사들이 한번에 일망타진한 기사를 접한 제작진은 바로 영화화 하기로 결정했다. 그 어떤 사건 현장보다도 위험했던 현장에 시민의 안전을 위해 맨몸을 던졌던 강력계 형사들의 이야기에 매료된 것. \n\n    강윤성 감독을 비롯한 제작진은 몇몇의 기사들을 바탕으로 영화적인 살을 붙여 나가기 시작했다. 특히, ‘실화’를 바탕으로 한 영화만의 매력인, 당시 시대적 배경과 사건 실체의 리얼리티를 강조하는 것에 더해 범죄를 소탕하는 과정의 쫀쫀한 재미를 영화적으로 그려 내는 것에 성공했다. 뿐만 아니라 실제 사건의 인물을 표

In [ ]:
# 주어진 질문에 대해 연속적으로 답변을 생성하고 출력하는 작업
for chunk in rag_chain.stream("마동석 나온 영화 뭐더라?"):
    print(chunk, end="", flush=True)

In [ ]:
response = rag_chain.invoke("마동석 나온 영화 뭐더라?")
print(response)

OutOfMemoryError: CUDA out of memory. Tried to allocate 5.16 GiB. GPU 0 has a total capacty of 39.56 GiB of which 812.81 MiB is free. Process 26945 has 38.76 GiB memory in use. Of the allocated memory 33.78 GiB is allocated by PyTorch, and 4.48 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF